In [1]:
%run -i rename.py

1636237312 bytes of graphical memory available.
11551571968 bytes of graphical memory available.


In [5]:
w1 = torch.load('../model/vsr/IconVSR_Vimeo90K_BDx4-cfcb7e00.pth')
pp(w1['params'])

edvr.conv_first.weight torch.Size([64, 3, 3, 3])
edvr.conv_first.bias torch.Size([64])
edvr.feature_extraction.0.conv1.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.0.conv1.bias torch.Size([64])
edvr.feature_extraction.0.conv2.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.0.conv2.bias torch.Size([64])
edvr.feature_extraction.1.conv1.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.1.conv1.bias torch.Size([64])
edvr.feature_extraction.1.conv2.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.1.conv2.bias torch.Size([64])
edvr.feature_extraction.2.conv1.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.2.conv1.bias torch.Size([64])
edvr.feature_extraction.2.conv2.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.2.conv2.bias torch.Size([64])
edvr.feature_extraction.3.conv1.weight torch.Size([64, 64, 3, 3])
edvr.feature_extraction.3.conv1.bias torch.Size([64])
edvr.feature_extraction.3.conv2.weight torch.Size([64, 64, 3, 3])
edvr.fe

In [ ]:
from models import Net2x, Net3x, Net4x, NetDN
models = [
    # model Class, kwargs, root, *paths
    (Net2x, {}, 0, '../model/a2/model_new.pth', '../model/p2/model_new.pth'),
    (Net3x, {}, 0, '../model/a3/model_new.pth', '../model/p3/model_new.pth'),
    (Net4x, {}, 0, '../model/a4/model_new.pth', '../model/p4/model_new.pth'),
    (NetDN, {}, 0, '../model/dn_lite5/model_new.pth', '../model/dn_lite10/model_new.pth', '../model/dn_lite15/model_new.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    ((r'convt_F.\.',), None, None, '', '0.')
]

In [ ]:
from models import RRDBNet
models = [
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3, 'scale': 2}, 'params_ema', '../model/gan/RealESRGAN_x2plus.pth'),
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3}, 'params_ema', '../model/gan/RealESRGAN_x4plus.pth'),
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3, 'num_block': 6}, 'params_ema', '../model/gan/RealESRGAN_x4plus_anime_6B.pth')
]
subs = [
    (re.compile(r'(body.[\d]+.rdb[\d]+.)conv([\d]+)(\..*)'), lambda m: ''.join((m[1], 'conv.', str(int(m[2]) - 1), m[3])))
]

In [ ]:
from basicvsr_arch import IconVSR
models = [
  (IconVSR, {'num_in_ch': 3, 'num_out_ch': 3}, 'params', '../model/vsr/IconVSR_Vimeo90K_BDx4-cfcb7e00.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    ((r'[^\.]+_trunk.\.',), 'main.', '', None, None)
]

In [ ]:
m = renameByRules(models, subs=subs)

In [ ]:
m = m.to(dtype=config.dtype(), device=config.device()).eval()
opt = Option(models[-1][-1])
opt.scale = 4
opt.padding = 5
opt.align = 32
opt.ramCoef = 1 / 190000.
opt.squeeze = lambda x: x.squeeze(0)
opt.unsqueeze = lambda x: x.unsqueeze(0)
opt.modelCached = m

In [ ]:
original = readPic('../images/example1sn.png')
show(original)

In [ ]:
y = ensemble(opt)(original)
show(y)